In [2]:
from src.dataset_creation import *
import os
import pandas as pd
import json
%load_ext autoreload
%autoreload 2

dataset_dir = '/media/external/siddsuresh97/datasets/ecoset_leuven'

In [4]:
leuven_animals = read_leuven_norms_as_df('../../data/LeuvenNorms/ANIMALSexemplarfeaturesbig.txt')
leuven_artifacts = read_leuven_norms_as_df('../../data/LeuvenNorms/ARTIFACTSexemplarfeaturesbig.txt')
leuven_animals_names = list(leuven_animals.columns[2:])
leuven_artifacts_names = list(leuven_artifacts.columns[2:])

In [11]:
# In the dataset directory, there are 3 folders: train, val, test
# Each folder has the same concept folders
# Make a df with the concept names and the number of images in each folder - train val test

def get_df(dataset_dir):
    df = pd.DataFrame(columns = ['concept', 'train', 'val', 'test'])
    for folder in os.listdir(dataset_dir):
        for concept in os.listdir(os.path.join(dataset_dir, folder)):
            if concept not in df['concept'].values:
                df = df.append({'concept': concept, 'train': 0, 'val': 0, 'test': 0}, ignore_index = True)
            df.loc[df['concept'] == concept, folder] = len(os.listdir(os.path.join(dataset_dir, folder, concept)))
    return df

df = get_df(dataset_dir)

In [14]:
# export the df to csv
df.to_csv(os.path.join('../../', 'dataset_stats.csv'), index = False)

In [4]:
# open the csv file and calculate class weights using only the train set
# class weights are calculated as the inverse of the number of images in the train set
# this is done to balance the classes in the dataset
# class weights are then printed using the concept names as the keys
df = pd.read_csv('../../dataset_stats.csv')
df['class_weight'] = df['train'].sum() / (len(df['concept'])*df['train'])
# save the class weights to a json file with the concept names as the keys
class_weights = df.set_index('concept')['class_weight'].to_dict()
with open('../../class_weights.json', 'w') as f:
    json.dump(class_weights, f)

In [5]:
# dict of class weights where concept is the key and class weight is the value
class_weights = json.load(open('../../class_weights.json'))



In [6]:
class_weights

{'towel': 0.8943063698523465,
 'bottle': 0.938717502504878,
 'wasp': 1.1592674841911261,
 'glass': 0.6620412899482667,
 'kettle': 1.926358392762453,
 'llama': 0.9956094723536584,
 'spider': 0.8563806756405692,
 'anvil': 4.752863589031587,
 'iguana': 1.4186923187273857,
 'car': 0.45732922956766586,
 'violin': 0.9019050514262552,
 'submarine': 1.483247648171448,
 'beaver': 1.826089186610724,
 'donkey': 1.2737674418604652,
 'worm': 0.9302795923700026,
 'hammer': 1.1518486883824461,
 'bus': 1.0472411297866207,
 'sieve': 2.279594816104907,
 'cat': 0.3807361524920916,
 'wheelbarrow': 2.6794460751110107,
 'moth': 0.8588680883913925,
 'jar': 1.4148134607130936,
 'elephant': 0.868052880278153,
 'mosquito': 1.1518486883824461,
 'caterpillar': 0.9164808913098321,
 'hedgehog': 2.5397203595377373,
 'bumblebee': 2.5444033104158033,
 'squirrel': 0.8554958020348237,
 'snake': 0.8613699929351876,
 'axe': 0.9670040144934622,
 'sheep': 0.8794867614290444,
 'wolf': 1.0708081184807325,
 'guitar': 0.8711582

In [ ]:
import numpy as np
import cv2
 
from pathlib import Path
 
imageFilesDir = Path(r'/media/external/siddsuresh97/datasets/ecoset_leuven/train')
files = list(imageFilesDir.rglob('*.jpg')) + list(imageFilesDir.rglob('*.JPEG'))
 
len(files)
 
mean = np.array([0.,0.,0.])
stdTemp = np.array([0.,0.,0.])
std = np.array([0.,0.,0.])
 
numSamples = len(files)
 
for i in range(numSamples):
    im = cv2.imread(str(files[i]))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = im.astype(float) / 255.
     
    for j in range(3):
        mean[j] += np.mean(im[:,:,j])
       
mean = (mean/numSamples)
 
print(mean) #[0.50616427 0.48602325 0.43117783] 


In [ ]:
for i in range(numSamples):
   ...:     im = cv2.imread(str(files[i]))
   ...:     im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
   ...:     im = im.astype(float) / 255.
   ...:     for j in range(3):
   ...:         stdTemp[j] += ((im[:,:,j] - mean[j])**2).sum()/(im.shape[0]*im.shape[1])
   ...: 
   ...: std = np.sqrt(stdTemp/numSamples)
   ...: 
   ...: print(std)

   # [0.28661095 0.27966835 0.29607392]

In [4]:
 #load dataset_stats.csv
df = pd.read_csv('../../dataset_stats.csv')
# find the sume of the number of images in the train set
total_images = df['train'].sum()
total_images

356018